### Один из вариант решение задаи по Optiver - Trading at the Close с хорошим показателем Mean Absolute Error (MAE) 

# Введение
Этот блокнот  с нашей ансамблевой моделью LGBM и CatBoost с использованием метода отложенной кросс-валидации (holdout CV) и демонстрации процесса онлайн-обучения для непрерывного сбора данных общественного рейтинга и обновления модели (примерно через 40 дней после date_id 480).


Особая благодарность авторам следующих общедоступных блокнотов за их неоценимый вклад:

@lblhandsome, "⚡Optiver 🚀Robust Best ⚡ Single Model"

@zulqarnainali, "LGB Fine-tuned 🚀(Explained)"

@verracodeguacas, "🖐️-Fold CV 🚀"

@yekenot, "Feature Elimination by CatBoost"

@jirkaborovec, "📈Optiver📉: Feature Eng & Optuna🌪️LightGBM@GPU"

@yunchonggan,"Weights of the Synthetic Index"

Обратите внимание, что для запуска следует использовать GPU T4x2. GPU P100 может вызвать проблемы с памятью при обучении Catboost на платформе Kaggle. Подробности см. в этом блокноте https://www.kaggle.com/code/yekenot/feature-elimination-by-catboost


# libraries

In [ ]:
import gc  # Управление сборкой мусора для управления памятью
import os  # Функции, связанные с операционной системой
import time  # Функции, связанные со временем
import warnings  # Обработка предупреждений
from itertools import combinations  # Для создания комбинаций элементов
from warnings import simplefilter  # Упрощение обработки предупреждений

#  Импорт библиотек машинного обучения
import joblib  # Для сохранения и загрузки моделей
import xgboost as xgb
import lightgbm as lgb  # Фреймворк LightGBM для градиентного бустинга
import catboost as ctb
import numpy as np  # Численные операции
import pandas as pd  # Манипуляция и анализ данных
# from pandarallel import pandarallel
from sklearn.metrics import mean_absolute_error  # Метрика для оценки
from sklearn.model_selection import KFold, TimeSeriesSplit  # Техники кросс-валидации

# Отключение предупреждений для чистоты кода
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
# pandarallel.initialize(nb_workers=4)


max_lookback = np.nan  # Максимальный размер окна возврата (не указан)
split_day = 435  # День разделения для данных временных рядов
N_STOCKS = 200  # Количество акций
MAX_N_NEIGHBOURS = 10  # Максимальное количество соседей
NEIGHBOUR_CORR_THRESHOLD = 0.3

In [ ]:
# is_offline = True  # Флаг для режима онлайн/оффлайн
# is_train = False  # Флаг для режима обучения
# is_infer = False  # Флаг для режима вывода

# EDA 

In [ ]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")

# Удаление строк с отсутствующими значениями в столбце "target"
df = df.dropna(subset=["target"])
# Удаляем строки, где в столбце "target" есть пропущенные значения.

df = df.dropna(subset=["wap"])
# Удаляем строки, где в столбце "wap" есть пропущенные значения.

# Сброс индекса DataFrame и применение изменений на месте
df.reset_index(drop=True, inplace=True)
# Сбрасываем индекс, чтобы он шел от 0 до количества строк - 1, и делаем это изменение в самом DataFrame (inplace=True).

# Получение размера DataFrame (количество строк и столбцов)
df_shape = df.shape
# Сохраняем размер DataFrame (количество строк и столбцов) в переменной df_shape.

df_train = df
# Сохраняем обработанный DataFrame в переменной df_train для дальнейшего использования.


# Оптимизация

In [ ]:
# Функция для уменьшения использования памяти DataFrame Pandas
def reduce_mem_usage(df, verbose=0):
    """
    Проходит через все числовые столбцы DataFrame и изменяет их тип данных
    для уменьшения использования памяти.
    """
    
    # Расчет начального использования памяти DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # Проход по каждому столбцу в DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Проверка, не является ли тип данных столбца 'object' (т.е., числовой)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Проверка, является ли тип данных столбца целочисленным
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Проверка, является ли тип данных столбца типом с плавающей точкой
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    # Предоставление информации об оптимизации памяти, если 'verbose' истина
    if verbose:
        logger.info(f"Использование памяти исходного dataframe составляет {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Использование памяти после оптимизации: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Уменьшено на {decrease:.2f}%")

    # Возвращение DataFrame с оптимизированным использованием памяти
    return df


# параллельный расчет триплетного дисбаланса с помощью Numba

Numba - это библиотека, предназначенная для ускорения выполнения кода на языке Python путем компиляции функций в машинный код. Она часто используется для улучшения производительности при работе с числовыми вычислениями.

Чтобы провести параллельные вычисления с использованием Numba, вы можете воспользоваться декоратором @njit (для Just-In-Time компиляции) и указать атрибут parallel=True. Вот пример, как это может выглядеть:

python
'''

        import numpy as np
        from numba import njit, prange
        
    @njit(parallel=True)
    def calculate_triplet_imbalance(data):
        num_elements = data.shape[0]
        result = np.zeros(num_elements)

    for i in prange(num_elements):
        # Ваш код для вычисления триплетного дисбаланса
        # Используйте data[i] для доступа к элементам массива

    return result

    # Пример использования
    data_array = np.random.rand(1000, 3)  # Замените на ваш массив данных
    result_array = calculate_triplet_imbalance(data_array)
'''

In [ ]:
# Импорт библиотеки Numba для компиляции "на лету" (JIT) и параллельной обработки
from numba import njit, prange

# Функция для вычисления дисбаланса тройных комбинаций в параллельном режиме с использованием Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # Цикл по всем комбинациям из трех элементов
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # Цикл по строкам DataFrame
        for j in range(num_rows):
            # Находим максимальное, минимальное и среднее значение из трех значений
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # Предотвращение деления на ноль
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

In [ ]:
# 📊 Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):

            if df_values[j, a] < df_values[j, b]:
                min_val = df_values[j, a]
                max_val = df_values[j, b]
            else:
                max_val = df_values[j, a]
                min_val = df_values[j, b]

            if min_val < df_values[j, c]:
                if df_values[j, c] < max_val:
                    mid_val = df_values[j, c]
                else:
                    mid_val = max_val
                    max_val = df_values[j, c]
            else:
                mid_val = min_val
                min_val = df_values[j, c]
            
            # 🚫 Prevent division by zero
            if max_val == min_val:
                imbalance_features[j, i] = np.nan
            elif mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)
    
    return imbalance_features

In [ ]:
# Функция для расчета дисбаланса тройных комбинаций по данным о ценах и DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Преобразование DataFrame в массив numpy для совместимости с Numba
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Вычисление дисбаланса тройных комбинаций с использованием оптимизированной функции Numba
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Создание DataFrame из результатов
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

# новые признаки 

In [ ]:
# Функция для генерации признаков дисбаланса
def imbalance_features(df):

    stock_groups = df.groupby(["date_id", "seconds_in_bucket"])
    # Индексированный WAP
    df["wwap"] = df.stock_id.map(weights) * df.wap
    df["iwap"] = stock_groups["wwap"].transform(lambda x: x.sum())
    del df["wwap"]

    # Определение списков названий столбцов, связанных с ценами и размерами
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1 признаки
    # Расчет различных признаков с использованием функции Pandas eval
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["all_size"] = df.eval("matched_size + imbalance_size") # добавление
    df["imbalance_size_for_buy_sell"] = df.eval("imbalance_size * imbalance_buy_sell_flag")  # добавление
    
    cols = ['wap', 'imbalance_size_for_buy_sell', "bid_size", "ask_size"]
    for q in [0.25, 0.5, 0.75]:  # Испытание большего/разного количества q
        df[[f'{col}_quantile_{q}' for col in cols]] = stock_groups[cols].transform(lambda x: x.quantile(q)).astype(np.float32)
    
    # Создание признаков для попарных дисбалансов цен
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})").astype(np.float32)

    for c in combinations(sizes, 2):
        df[f"{c[0]}/{c[1]}"] = df.eval(f"({c[0]})/({c[1]})").astype(np.float32)

    # Расчет признаков дисбаланса тройных комбинаций с использованием оптимизированной функции Numba
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values.astype(np.float32)
        
    # V2 признаки
    # Расчет дополнительных признаков
    stock_groups = df.groupby(['stock_id', 'date_id'])
    df["imbalance_momentum"] = stock_groups['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = stock_groups['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['wap_advantage'] = df.wap - df.iwap  # добавление

    # Расчет различных статистических агрегированных признаков
    df_prices = df[prices]
    df_sizes = df[sizes]
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df_prices.agg(func, axis=1)
        df[f"all_sizes_{func}"] = df_sizes.agg(func, axis=1)
        
    # V3 признаки
    # Расчет сдвинутых и возвращаемых признаков для конкретных столбцов
    cols = ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag', "wap", "iwap"]
    stock_groups_cols = stock_groups[cols]
    for window in [1, 2, 3, 6, 10]:
        df[[f"{col}_shift_{window}" for col in cols]] = stock_groups_cols.shift(window)

    cols = ['matched_size', 'imbalance_size', 'reference_price', "iwap"] #wap
    stock_groups_cols = stock_groups[cols]
    for window in [1, 2, 3, 6, 10]:
        df[[f"{col}_ret_{window}" for col in cols]] = stock_groups_cols.pct_change(window).astype(np.float32)

    # Расчет признаков разницы для конкретных столбцов
    cols = ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'wap', 'near_price', 'far_price', 'imbalance_size_for_buy_sell']
    stock_groups_cols = stock_groups[cols]
    for window in [1, 2, 3, 6, 10]:
        df[[f"{col}_diff_{window}" for col in cols]] = stock_groups_cols.diff(window).astype(np.float32)

    # V4 признаки
    # Построение признака `time_since_last_imbalance_change`
    df['flag_change'] = stock_groups['imbalance_buy_sell_flag'].diff().ne(0).astype(int)
    # Использование cumsum для создания идентификатора группы, увеличивающегося при каждом изменении флага
    df['group'] = df.groupby(['stock_id', 'date_id'])['flag_change'].cumsum()
    # Расчет временного интервала с момента последнего изменения флага в каждой группе
    group_min = df.groupby(['stock_id', 'date_id', 'group'])['seconds_in_bucket'].transform('min')
    df['time_since_last_imbalance_change'] = df['seconds_in_bucket'] - group_min
    # Обнуление времени в местах, где произошло изменение флага
    df['time_since_last_imbalance_change'] *= (1 - df['flag_change'])
    df.drop(columns=['flag_change', 'group'], inplace=True)
    
    cols = ['imbalance_size_for_buy_sell']
    stock_groups_cols = stock_groups[cols]
    for window in [5, 10]:
        mean_col = stock_groups_cols.transform(lambda x: x.rolling(window=window).mean())
        std_col = stock_groups_cols.transform(lambda x: x.rolling(window=window).std())
        df[[f'z_score_{col}_{window}' for col in cols]] = (df[cols] - mean_col) / std_col
    
    # Замена бесконечных значений на 0
    return df.replace([np.inf, -np.inf], 0)

# Функция для генерации временных и акционных признаков
def other_features(df):
    df["seconds"] = df["seconds_in_bucket"] % 60  # Секунды
    df["minute"] = df["seconds_in_bucket"] // 60  # Минуты

    # Присвоение глобальных признаков к DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
        
    for key, value in global_seconds_feats.items():
        df[f"global_seconds_{key}"] = df["seconds_in_bucket"].map(value.to_dict())

    return df

           


добавление признаков последних дней, включая подготовку данных, сдвиг столбцов для создания признаков и объединение различных наборов признаков. В функции generate_all_features применяются различные методы для создания комплексного набора признаков, используемого в анализе данных.

In [ ]:
def last_days_features(df: pd.DataFrame, feat_last=None, target_last=None):
    size = None
    
    # Если предыдущие признаки предоставлены и их количество больше нуля
    if feat_last is not None and len(feat_last) > 0:
        # Выбор столбцов, которые присутствуют как в текущем, так и в предыдущем DataFrame
        cols = [col for col in df.columns if col in set(feat_last.columns)]
        # Если предоставлены целевые значения для предыдущих признаков
        if target_last is not None:
            # Добавление целевого столбца и заполнение его значениями
            cols.append("target")
            feat_last["target"] = target_last
            df["target"] = 0
        # Подготовка к добавлению заполнителей
        paddings = []
        # Нахождение максимального значения секунд в текущем DataFrame
        second_start = df.seconds_in_bucket.max()
        # Получение источника данных для заполнителей
        padding_src = df[df.seconds_in_bucket == second_start]
        # Определение размеров текущего и заполнительного DataFrame
        size = len(df)
        size_pad = len(padding_src) * 6
        # Добавление заполнителей для каждого шага в 10 секунд
        for second in range(second_start + 10, second_start + 70, 10):
            padding = padding_src.copy()
            padding["seconds_in_bucket"] = second
            paddings.append(padding)
        # Объединение предыдущих признаков, текущих данных и заполнителей
        df = pd.concat([feat_last[cols], df] + paddings)

    # Добавление признаков последних дней
    # TODO: Попробовать больше признаков
    cols = ['near_price', 'far_price', 'depth_pressure']
    if 'target' in df.columns:
        cols.append('target')
    stock_groups = df.groupby(['stock_id', 'seconds_in_bucket'])
    stock_groups_cols = stock_groups[cols]
    # Сдвиг столбцов для создания признаков
    for window in [1]:
        df[[f"{col}_last_{window}day" for col in cols]] = stock_groups_cols.shift(window)
    if cols[-1] == "target":
        cols.pop()
    
    cols = [f"{col}_last_{window}day" for col in cols]
    stock_groups = df.groupby(['stock_id', 'date_id'])
    stock_groups_cols = stock_groups[cols]
    # Сдвиг столбцов для создания будущих признаков
    for window in [1, 2, 3, 6]:
        df[[f"{col}_future_{window}" for col in cols]] = stock_groups_cols.shift(-window)
        
    if size:
        return df[-(size + size_pad):-size_pad]
    return df

# Функция для генерации всех признаков путем комбинирования признаков дисбаланса и других признаков
def generate_all_features(df, feat_last=None, target_last=None):
    # Выбор релевантных столбцов для генерации признаков
    cols = [c for c in df.columns if c not in {"row_id", "time_id", "currently_scored"}]
    df = df[cols]
    
    # Генерация признаков дисбаланса
    df = imbalance_features(df)
    
    # Генерация признаков последних дней
    df = last_days_features(df, feat_last, target_last)
    
    # Генерация временных и акционных признаков
    df = other_features(df)

    gc.collect()  # Осуществление сборки мусора для освобождения памяти
    
    # Выбор и возврат сгенерированных признаков
    feature_name = [i for i in df.columns if i not in {"row_id", "target", "time_id"}]
    
    return df[feature_name]


## ВЕса 

In [ ]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

weights = {int(k):v for k,v in enumerate(weights)}

оздание глобальных признаков на основе группировки данных по идентификатору акций и по времени в секундах

In [ ]:
# Группировка данных обучающего набора по идентификатору акций
stock_group = df_train.groupby("stock_id")

# Создание нового столбца "imbalance_size_for_buy_sell" путем умножения размера дисбаланса на флаг покупки/продажи
df_train["imbalance_size_for_buy_sell"] = df_train.eval("imbalance_size * imbalance_buy_sell_flag")

# Глобальные признаки по идентификатору акций
global_stock_id_feats = {
    "median_size": stock_group["bid_size"].median() + stock_group["ask_size"].median(),  # Медиана размера заявок покупки и продажи
    "std_size": stock_group["bid_size"].std() + stock_group["ask_size"].std(),  # Стандартное отклонение размера заявок покупки и продажи
    "ptp_size": stock_group["bid_size"].max() - stock_group["bid_size"].min(),  # Размах размера заявок покупки
    "median_price": stock_group["bid_price"].median() + stock_group["ask_price"].median(),  # Медиана цен заявок покупки и продажи
    "std_price": stock_group["bid_price"].std() + stock_group["ask_price"].std(),  # Стандартное отклонение цен заявок покупки и продажи
    "ptp_price": stock_group["bid_price"].max() - stock_group["ask_price"].min(),  # Размах цен заявок покупки и продажи
    "median_far_price": stock_group["far_price"].median(),  # Медиана дальней цены
    "median_near_price": stock_group["near_price"].median(),  # Медиана ближней цены
    "median_imbalance_size_for_buy_sell": stock_group["imbalance_size_for_buy_sell"].median(),  # Медиана дисбаланса размера для покупки/продажи
    "matched_size":stock_group["matched_size"].median(),  # Медиана согласованного размера
}

# Глобальные признаки по секундам
global_seconds_feats = {
    # Медиана целевого значения по секундам внутри каждой даты, а затем в целом по секундам
    "median_target": df_train.groupby(["date_id", "seconds_in_bucket"])["target"].apply(lambda x: x.abs().mean()).reset_index(0, drop=True).groupby("seconds_in_bucket").median(),
}


# LGBM 


LGBM параметры из  **@jirkaborovec**, ["📈Optiver📉: Feature Eng & Optuna🌪️LightGBM@GPU"](https://www.kaggle.com/code/jirkaborovec/optiver-feature-eng-optuna-lightgbm-gpu) 

In [ ]:
def train_lgb(df_train_feats, targets, split_day):

    feature_name = list(df_train_feats.columns)
    feature_name.remove("date_id")
    offline_split = df_train_feats['date_id'] > (split_day - 45)
    

    stock_group = df_train_feats.groupby("stock_id")
    global global_stock_id_feats, global_seconds_feats
    global_stock_id_feats = {
        "median_size": stock_group["bid_size"].median() + stock_group["ask_size"].median(),
        "std_size": stock_group["bid_size"].std() + stock_group["ask_size"].std(),
        "ptp_size": stock_group["bid_size"].max() - stock_group["bid_size"].min(),
        "median_price": stock_group["bid_price"].median() + stock_group["ask_price"].median(),
        "std_price": stock_group["bid_price"].std() + stock_group["ask_price"].std(),
        "ptp_price": stock_group["bid_price"].max() - stock_group["ask_price"].min(),
        "median_far_price": stock_group["far_price"].median(),
        "median_near_price": stock_group["near_price"].median(),
        "median_imbalance_size_for_buy_sell": stock_group["imbalance_size_for_buy_sell"].median(),
        "matched_size": stock_group["matched_size"].median(),
    }
    df_train_feats["target"] = targets.values
    global_seconds_feats = {
        "median_target": df_train_feats.groupby(["date_id", "seconds_in_bucket"])["target"].apply(lambda x: x.abs().mean()).reset_index(0, drop=True).groupby("seconds_in_bucket").median(),
    }
    del df_train_feats["target"]
    
    # update global features
    for key, value in global_stock_id_feats.items():
        df_train_feats[f"global_{key}"] = df_train_feats["stock_id"].map(value.to_dict())
        
    for key, value in global_seconds_feats.items():
        df_train_feats[f"global_seconds_{key}"] = df_train_feats["seconds_in_bucket"].map(value.to_dict())
        
    lgb_params = {
        "objective": "mae",
        "n_estimators": 5500,
        "num_leaves": 465,
        "subsample": 0.65791,
        "colsample_bytree": 0.7,
        "learning_rate": 0.00877,  # 0.00877
        "n_jobs": 4,
        "device": "gpu",
        "verbosity": -1,
        "importance_type": "gain",
        "max_depth": 14,  # Maximum depth of the tree
        "min_child_samples": 132,  # Minimum number of data points in a leaf
        "reg_alpha": 6,  # L1 regularization term
        "reg_lambda": 0.08,  # L2 regularization term
    }

    print(f"Feature length = {len(feature_name)}")

    # Split data for offline training based on a specific date
    df_offline_train = df_train_feats[~offline_split]
    df_offline_valid = df_train_feats[offline_split]
    df_offline_train_target = targets[~offline_split]
    df_offline_valid_target = targets[offline_split]
    
    
    train_dataset = lgb.Dataset(df_offline_train[feature_name].values.astype(np.float32), 
                                label=df_offline_train_target.values.astype(np.float32))
    
    valid_dataset = lgb.Dataset(df_offline_valid[feature_name].values.astype(np.float32), 
                                label=df_offline_valid_target.values.astype(np.float32))
    
    del df_offline_train, df_offline_valid, df_offline_train_target, df_offline_valid_target, offline_split

    print("Valid Model Training.")
    
    # Train a LightGBM model on the offline data
    lgb_model = lgb.LGBMRegressor(**lgb_params)
  
    lgb_model.fit(
        train_dataset.data,
        train_dataset.label,
        eval_set=[(valid_dataset.data, valid_dataset.label)],
        callbacks=[
            lgb.callback.early_stopping(stopping_rounds=200),
            lgb.callback.log_evaluation(period=100),
        ],
        feature_name=feature_name,
    )
    
    best_iteration_ = lgb_model.best_iteration_
    
    # Free up memory by deleting variables
    del lgb_model
    gc.collect()

    # Inference
    df_train_target = targets
    full_train_dataset = lgb.Dataset(df_train_feats[feature_name].values.astype(np.float32), 
                                     label=df_train_target.values.astype(np.float32))
    print("Infer Model Training.")

    # Adjust the number of estimators for the inference model
    infer_params = lgb_params.copy()
    infer_params["n_estimators"] = int(1.2 * best_iteration_)
    infer_lgb_model = lgb.LGBMRegressor(**infer_params)
    
    infer_lgb_model.fit(
        full_train_dataset.data,
        full_train_dataset.label,
        feature_name=feature_name,
        )
    
#     infer_lgb_model.fit(
#         df_train_feats[feature_name].values.astype(np.float32), 
#         df_train_target.values.astype(np.float32), 
#         feature_name = feature_name
#         )

    return infer_lgb_model

# Catboost тренировка 

Спасибо  **@yekenot**, ["Feature Elimination by CatBoost"](https://www.kaggle.com/code/yekenot/feature-elimination-by-catboost)


In [ ]:
eliminated_features_names = set(['near_price_ask_price_imb',
                             'iwap',
                             'all_imbalance_size_for_buy_sell_quantile_0.5',
                             'iwap_shift_10',
                             'all_imbalance_size_for_buy_sell_quantile_0.75',
                             'matched_size_ret_10',
                             'all_wap_quantile_0.25',
                             'iwap_shift_6',
                             'near_price_wap_imb',
                             'iwap_shift_1',
                             'iwap_ret_10',
                             'iwap_ret_6',
                             'all_imbalance_size_for_buy_sell_quantile_0.25',
                             'global_ptp_size',
                             'reference_price_shift_10',
                             'all_ask_size_quantile_0.75',
                             'iwap_shift_3',
                             'iwap_shift_2',
                             'reference_price_near_price_imb',
                             'all_bid_size_quantile_0.25',
                             'global_std_size',
                             'global_median_price',
                             'matched_size_ret_6',
                             'wap_shift_10'])

In [ ]:
def train_ctb(df_train_feats, targets, split_day):

    feature_name = list(df_train_feats.columns)
    feature_name.remove("date_id")
    feature_name = [feat for feat in feature_name if feat not in eliminated_features_names]
    offline_split = df_train_feats['date_id'] > (split_day - 45)
    
#     target_mean = targets.values.mean()
#     pre = df_train_feats[offline_split][["date_id", "seconds_in_bucket"]]
#     pre["target"] = targets[offline_split].values
    
    ctb_params = dict(iterations=2000,
                      learning_rate=1.0,
                      depth=9,
                      l2_leaf_reg=30,
                      bootstrap_type='Bernoulli',
                      subsample=0.66,
                      loss_function='MAE',
                      eval_metric = 'MAE',
                      metric_period=100,
                      od_type='Iter',
                      od_wait=200,
                      task_type='GPU',
                      allow_writing_files=False,
                      random_strength=4.428571428571429
                      )

    print(f"Feature length = {len(feature_name)}")

    # Split data for offline training based on a specific date
    df_offline_train = df_train_feats[~offline_split]
    df_offline_valid = df_train_feats[offline_split]
    df_offline_train_target = targets[~offline_split]
    df_offline_valid_target = targets[offline_split]
    
#     train_dataset = lgb.Dataset(df_offline_train[feature_name].values, label = df_offline_train_target, feature_name = feature_name)
#     valid_dataset = lgb.Dataset(df_offline_valid[feature_name].values, label = df_offline_valid_target, feature_name = feature_name)

    print("Valid Model Training.")

    # Train a LightGBM model on the offline data
    ctb_model = ctb.CatBoostRegressor(**ctb_params)
    
#   feature elinmation 

#     summary = ctb_model.select_features(
#         df_offline_train[feature_name], df_offline_train_target,
#         eval_set=[(df_offline_valid[feature_name], df_offline_valid_target)],
#         features_for_select=feature_name,
#         num_features_to_select=len(feature_name)-24,    # Dropping from 124 to 100
#         steps=3,
#         algorithm=ctb.EFeaturesSelectionAlgorithm.RecursiveByShapValues,
#         shap_calc_type=ctb.EShapCalcType.Regular,
#         train_final_model=False,
#         plot=True,
#     )
    ctb_model.fit(
        df_offline_train[feature_name].astype(np.float32), df_offline_train_target.astype(np.float32),
        eval_set=[(df_offline_valid[feature_name].astype(np.float32), df_offline_valid_target.astype(np.float32))],
        use_best_model=True,
#         early_stopping_rounds=200
    )
    
    best_iteration_ = ctb_model.best_iteration_
    
    # Free up memory by deleting variables
    del df_offline_train, df_offline_valid, df_offline_train_target, df_offline_valid_target, offline_split, ctb_model
    gc.collect()

    # Inference
    df_train_target = targets
    print("Infer Model Training.")

    # Adjust the number of estimators for the inference model
    infer_params = ctb_params.copy()
    infer_params["iterations"] = int(1.2 * best_iteration_)
    infer_ctb_model = ctb.CatBoostRegressor(**infer_params)
    infer_ctb_model.fit(df_train_feats[feature_name].astype(np.float32), df_train_target.astype(np.float32))
    return infer_ctb_model

# Логический вывод и онлайн-обучение

Обучение модели по дате_id 481, 481+15, 481+30 во время логического вывода

In [ ]:
def zero_sum(prices, volumes):
    # Расчет стандартной ошибки для объемов
    std_error = np.sqrt(volumes)
    # Вычисление шага на основе суммы цен и стандартных ошибок
    step = np.sum(prices)/np.sum(std_error)
    # Корректировка цен, учитывая стандартную ошибку и шаг
    out = prices-std_error*step
    
    return out

import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()
counter = 0
y_min, y_max = -64, 64
qps, predictions = [], []
cache = pd.DataFrame()
# train_cache = pd.DataFrame()
print("Начало инициализации...")
targets = df_train["target"].astype(np.float32)
# Уменьшение использования памяти и генерация всех признаков
data = reduce_mem_usage(generate_all_features(df_train))
del df_train
gc.collect()
print("Инициализация завершена.")
start_date_id = 481  # Начальный date_id
start_pro_date_id = 480
# Онлайн обучение
train_date_id = set([start_date_id, start_date_id + 15, start_date_id + 30])  # Набор date_id для обучения
feature_name = list(data.columns)
feature_name.remove("date_id")
# Отфильтрованные признаки для CatBoost
feature_name_ctb = [feat for feat in feature_name if feat not in eliminated_features_names]
feat_last = None
target_last = None
# Среднее значение целевой переменной
target_mean = targets.values.mean()
print("Начало предсказания...")
for (test, revealed_targets, sample_prediction) in iter_test:
    # Проверка на текущую оценку
    currently_scored = test.iloc[0]["currently_scored"]
    date_id = test.iloc[0]["date_id"]
    if not currently_scored and date_id != start_pro_date_id:
        sample_prediction['target'] = 0
        env.predict(sample_prediction)
        continue
    del test['currently_scored']
    seconds_in_bucket = test.iloc[0]["seconds_in_bucket"]
    # Онлайн обучение, если это начало нового торгового периода
    if seconds_in_bucket == 0:
        cache = pd.DataFrame()
        target_last = revealed_targets["revealed_target"].values.astype(np.float32)
        #concat
        if len(targets) < len(data):
            targets = pd.concat([targets, revealed_targets["revealed_target"].astype(np.float32)], ignore_index=True, axis=0)

        if date_id in train_date_id:
            infer_lgb_model = train_lgb(data, targets, date_id - 1)
            infer_ctb_model = train_ctb(data, targets, date_id - 1)
            target_mean = targets.values.mean

    # Обработка данных тестового набора
    cache = pd.concat([cache, test], ignore_index=True, axis=0)
    feat = generate_all_features(cache, feat_last, target_last)
    if seconds_in_bucket == 540:
        # Запоминание последних признаков для следующего цикла
        feat_last = feat.copy()
        if currently_scored:
            # Объединение данных для онлайн обучения
            data = pd.concat([data, reduce_mem_usage(feat)], ignore_index=True, axis=0)
    if not currently_scored:
        sample_prediction['target'] = 0
        env.predict(sample_prediction)
        continue
    # Обрезание признаков для текущего тестового набора
    feat = feat[-len(test):]
    # Предсказание с использованием моделей
    lgb_prediction = infer_lgb_model.predict(feat[feature_name])
    ctb_prediction = infer_ctb_model.predict(feat[feature_name_ctb])
    prediction = lgb_prediction * 0.67 + ctb_prediction * 0.33
    prediction = prediction - prediction.mean() + target_mean
    clipped_predictions = np.clip(prediction, y_min, y_max)
    sample_prediction['target'] = clipped_predictions
    env.predict(sample_prediction)
print("OK")
